# PhysiCell Settings Package - Comprehensive Demo

This notebook demonstrates the main features of the **PhysiCell Settings** package, a powerful Python library for generating PhysiCell XML configuration files with comprehensive parameter coverage and PhysiBoSS integration.

## Features Demonstrated:
- 🏗️ **Domain Setup** - Simulation bounds and mesh configuration
- 🧪 **Substrates** - Microenvironment with diffusion and decay
- 🦠 **Cell Types** - Multiple cell types with different properties
- 🧬 **PhysiBoSS/MaBoSS** - Boolean network integration for intracellular models
- 📋 **Cell Rules** - Behavior control based on environmental signals
- 📄 **File Export** - Generate cell rules CSV and PhysiCell XML files
- ✅ **Validation** - Configuration checking and summary

## Package Information:
- **Version**: 0.3.3
- **License**: GPL v3
- **Repository**: https://github.com/mruscone/PhysiCell_Settings
- **PyPI**: https://pypi.org/project/physicell-settings/

## 1. Install and Import PhysiCell Settings Package

First, we'll install the package (if needed) and import all necessary libraries.

In [1]:
# Install the PhysiCell Settings package (uncomment if not already installed)
# !pip install physicell-settings

# Import the main PhysiCell configuration class
import sys
import os
sys.path.append('/home/mruscone/Desktop/github/physicell_config')

from physicell_config import PhysiCellConfig

# Import additional Python libraries for demonstration
import json
from datetime import datetime

print("✅ PhysiCell Settings package imported successfully!")
print(f"📦 Package location: {PhysiCellConfig.__module__}")
print(f"⏰ Demo started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ PhysiCell Settings package imported successfully!
📦 Package location: physicell_config.config_builder_modular
⏰ Demo started at: 2025-08-08 17:57:04


## 2. Initialize Configuration and Set Domain

Create a new PhysiCell configuration and set up the simulation domain with appropriate bounds and mesh spacing.

In [2]:
# Create a new PhysiCell configuration instance
config = PhysiCellConfig()

print("🏗️ Setting up simulation domain...")

# Set domain bounds (in microns)
config.domain.set_bounds(
    x_min=-400.0, x_max=400.0,  # 800 microns wide
    y_min=-400.0, y_max=400.0,  # 800 microns tall
    z_min=-10.0, z_max=10.0     # 20 microns thick (2D simulation)
)

# Set mesh spacing (voxel size)
config.domain.set_mesh(
    dx=20.0,  # 20 micron spacing in x
    dy=20.0,  # 20 micron spacing in y
    dz=20.0   # 20 micron spacing in z
)

# Enable 2D simulation mode
config.domain.set_2D(True)

# Set simulation time parameters
config.options.set_max_time(2880.0)  # 48 hours (2880 minutes)
config.options.set_time_steps(
    dt_diffusion=0.01,    # Fast diffusion time step
    dt_mechanics=0.1,     # Mechanics time step
    dt_phenotype=6.0      # Phenotype update every 6 minutes
)

# Display domain information
domain_info = config.domain.get_info()
print(f"✅ Domain configured:")
print(f"   📏 Bounds: X=[{domain_info['x_min']}, {domain_info['x_max']}], Y=[{domain_info['y_min']}, {domain_info['y_max']}]")
print(f"   🔲 Mesh spacing: {domain_info['dx']} × {domain_info['dy']} × {domain_info['dz']} microns")
print(f"   📐 2D simulation: {domain_info['use_2D']}")
print(f"   ⏱️ Max time: {config.options.get_options()['max_time']} minutes")

🏗️ Setting up simulation domain...
✅ Domain configured:
   📏 Bounds: X=[-400.0, 400.0], Y=[-400.0, 400.0]
   🔲 Mesh spacing: 20.0 × 20.0 × 20.0 microns
   📐 2D simulation: True
   ⏱️ Max time: 2880.0 minutes


## 3. Add Substrates to Microenvironment

Configure multiple substrates with different diffusion coefficients, decay rates, and boundary conditions.

In [3]:
print("🧪 Adding substrates to microenvironment...")

# 1. Oxygen - essential for cellular respiration
config.substrates.add_substrate(
    name="oxygen",
    diffusion_coefficient=100000.0,  # Fast diffusion (micron²/min)
    decay_rate=0.1,                  # Moderate consumption rate (1/min)
    initial_condition=38.0,          # Initial concentration (mmHg)
    dirichlet_enabled=True,          # Fixed boundary conditions
    dirichlet_value=38.0,            # Boundary value (atmospheric)
    units="mmHg",
    initial_units="mmHg"
)

# Set specific boundary conditions for oxygen
for boundary in ['xmin', 'xmax', 'ymin', 'ymax']:
    config.substrates.set_dirichlet_boundary("oxygen", boundary, True, 38.0)

# 2. Glucose - primary energy source
config.substrates.add_substrate(
    name="glucose",
    diffusion_coefficient=50000.0,   # Moderate diffusion
    decay_rate=0.005,                # Slow decay rate
    initial_condition=5.0,           # Initial concentration
    dirichlet_enabled=False,         # No fixed boundaries
    dirichlet_value=0.0,
    units="mM",
    initial_units="mM"
)

# 3. Growth factor - signaling molecule
config.substrates.add_substrate(
    name="growth_factor",
    diffusion_coefficient=25000.0,   # Slower diffusion (larger molecule)
    decay_rate=0.01,                 # Faster decay
    initial_condition=0.0,           # Start with no growth factor
    dirichlet_enabled=False,
    dirichlet_value=0.0,
    units="ng/mL",
    initial_units="ng/mL"
)

# 4. Apoptotic signal - death-inducing factor
config.substrates.add_substrate(
    name="apoptotic_signal",
    diffusion_coefficient=10000.0,   # Slow diffusion
    decay_rate=0.05,                 # Fast decay
    initial_condition=0.0,
    dirichlet_enabled=False,
    dirichlet_value=0.0,
    units="dimensionless",
    initial_units="dimensionless"
)

# Enable tracking of internalized substrates
config.substrates.set_track_internalized_substrates(True)

# Display substrate information
substrates = config.substrates.get_substrates()
print(f"✅ Added {len(substrates)} substrates:")
for name, substrate in substrates.items():
    print(f"   🔬 {name}: D={substrate['diffusion_coefficient']:.0f} μm²/min, "
          f"decay={substrate['decay_rate']:.3f} min⁻¹, "
          f"initial={substrate['initial_condition']} {substrate['initial_units']}")

🧪 Adding substrates to microenvironment...
✅ Added 4 substrates:
   🔬 oxygen: D=100000 μm²/min, decay=0.100 min⁻¹, initial=38.0 mmHg
   🔬 glucose: D=50000 μm²/min, decay=0.005 min⁻¹, initial=5.0 mM
   🔬 growth_factor: D=25000 μm²/min, decay=0.010 min⁻¹, initial=0.0 ng/mL
   🔬 apoptotic_signal: D=10000 μm²/min, decay=0.050 min⁻¹, initial=0.0 dimensionless


## 4. Create Cell Types with Different Properties

Define multiple cell types with different cycle models, motility parameters, and secretion profiles.

In [4]:
print("🦠 Creating cell types with different properties...")

# 1. Cancer Cell - will have PhysiBoSS integration
config.cell_types.add_cell_type("cancer_cell", template="default")

# Set cancer cell cycle model
config.cell_types.set_cycle_model("cancer_cell", "Ki67_basic")

# Configure cancer cell motility (moderately motile)
config.cell_types.set_motility(
    "cancer_cell",
    speed=0.8,              # Moderate migration speed (micron/min)
    persistence_time=15.0,   # How long to keep direction (min)
    migration_bias=0.2,      # Slight directional bias
    enabled=True
)

# Cancer cells consume oxygen and glucose heavily
config.cell_types.add_secretion("cancer_cell", "oxygen", 
                               secretion_rate=0.0, uptake_rate=20.0)
config.cell_types.add_secretion("cancer_cell", "glucose", 
                               secretion_rate=0.0, uptake_rate=10.0)

# Cancer cells secrete growth factor when healthy
config.cell_types.add_secretion("cancer_cell", "growth_factor", 
                               secretion_rate=0.1, secretion_target=1.0)

# Set volume parameters
config.cell_types.set_volume_parameters("cancer_cell", total=2494, nuclear=540, fluid_fraction=0.75)

print("   ✅ Cancer cell configured with Ki67_basic cycle model")

# 2. Immune Cell - T cell that attacks cancer
config.cell_types.add_cell_type("immune_cell", template="live_cell")

# Set immune cell cycle (live cells don't proliferate much)
config.cell_types.set_cycle_model("immune_cell", "live")

# Configure immune cell motility (highly motile)
config.cell_types.set_motility(
    "immune_cell",
    speed=2.0,               # Fast migration
    persistence_time=5.0,    # Change direction frequently
    migration_bias=0.8,      # Strong directional bias when tracking
    enabled=True
)

# Immune cells consume moderate oxygen
config.cell_types.add_secretion("immune_cell", "oxygen", 
                               secretion_rate=0.0, uptake_rate=5.0)

# Immune cells secrete apoptotic signals when activated
config.cell_types.add_secretion("immune_cell", "apoptotic_signal", 
                               secretion_rate=0.05, secretion_target=1.0)

# Set smaller volume for immune cells
config.cell_types.set_volume_parameters("immune_cell", total=1500, nuclear=300, fluid_fraction=0.8)

print("   ✅ Immune cell configured with live cycle model")

# 3. Stromal Cell - supportive tissue
config.cell_types.add_cell_type("stromal_cell", template="default")

# Stromal cells have slow cycling
config.cell_types.set_cycle_model("stromal_cell", "cycling_quiescent")

# Stromal cells are mostly stationary
config.cell_types.set_motility(
    "stromal_cell",
    speed=0.1,               # Very slow migration
    persistence_time=60.0,   # Stay in direction for long time
    migration_bias=0.0,      # No directional bias
    enabled=True
)

# Stromal cells consume little and secrete growth factor
config.cell_types.add_secretion("stromal_cell", "oxygen", 
                               secretion_rate=0.0, uptake_rate=2.0)
config.cell_types.add_secretion("stromal_cell", "growth_factor", 
                               secretion_rate=0.2, secretion_target=2.0)

# Larger volume for stromal cells
config.cell_types.set_volume_parameters("stromal_cell", total=3000, nuclear=600, fluid_fraction=0.7)

print("   ✅ Stromal cell configured with cycling_quiescent model")

# Update all cell types to include secretion for all substrates
config.cell_types.update_all_cell_types_for_substrates()

# Display cell type information
cell_types = config.cell_types.get_cell_types()
print(f"\n✅ Created {len(cell_types)} cell types:")
for name in cell_types.keys():
    cycle_model = cell_types[name]['phenotype']['cycle'].get('model', 'unknown')
    motility_enabled = cell_types[name]['phenotype']['motility'].get('is_motile', False)
    speed = cell_types[name]['phenotype']['motility'].get('speed', 0)
    print(f"   🔬 {name}: cycle={cycle_model}, motile={motility_enabled}, speed={speed} μm/min")

🦠 Creating cell types with different properties...
   ✅ Cancer cell configured with Ki67_basic cycle model
   ✅ Immune cell configured with live cycle model
   ✅ Stromal cell configured with cycling_quiescent model

✅ Created 3 cell types:
   🔬 cancer_cell: cycle=Ki67_basic, motile=False, speed=0.8 μm/min
   🔬 immune_cell: cycle=live, motile=False, speed=2.0 μm/min
   🔬 stromal_cell: cycle=cycling_quiescent, motile=False, speed=0.1 μm/min


## 5. Configure PhysiBoSS Integration with MaBoSS

Add intracellular MaBoSS boolean network models to simulate gene regulatory networks within cells.

In [6]:
print("🧬 Configuring PhysiBoSS/MaBoSS integration for cancer cells...")
#from physicell_config.modules.physiboss import PhysiBoSSModule as PhysiBoSS


#physiboss_config = PhysiBoSS(config)

# Add intracellular boolean network model to cancer cells
config.physiboss.add_intracellular_model(
    cell_type_name="cancer_cell",
    model_type="maboss",
    bnd_filename="config/cancer_network.bnd",  # Boolean network definition
    cfg_filename="config/cancer_network.cfg"   # Configuration file
)

# Configure intracellular model settings
config.physiboss.set_intracellular_settings(
    cell_type_name="cancer_cell",
    intracellular_dt=6.0,         # Time step for boolean network (minutes)
    time_stochasticity=0,         # Deterministic simulation
    scaling=1.0,                  # Time scaling factor
    start_time=0.0,               # Start time for intracellular model
    inheritance_global=False      # Cell-specific inheritance
)

print("   ✅ MaBoSS model added to cancer cells")

# Add input mappings (PhysiCell → Boolean Network)
print("   🔗 Adding input mappings (PhysiCell → Boolean Network)...")

# Oxygen level affects p53 tumor suppressor
config.physiboss.add_intracellular_input(
    cell_type_name="cancer_cell",
    physicell_name="oxygen",
    intracellular_name="p53",
    action="activation",
    threshold=20.0,               # Threshold oxygen level
    smoothing=0
)

# Growth factor affects EGFR (growth receptor)
config.physiboss.add_intracellular_input(
    cell_type_name="cancer_cell",
    physicell_name="growth_factor",
    intracellular_name="EGFR",
    action="activation",
    threshold=0.5,
    smoothing=0
)

# Contact with immune cells affects DNA damage pathways
config.physiboss.add_intracellular_input(
    cell_type_name="cancer_cell",
    physicell_name="contact with immune_cell",
    intracellular_name="DNA_damage",
    action="activation",
    threshold=0.1,
    smoothing=0
)

print("   ✅ Added 3 input mappings")

# Add output mappings (Boolean Network → PhysiCell)
print("   🔗 Adding output mappings (Boolean Network → PhysiCell)...")

# p53 activation leads to apoptosis
config.physiboss.add_intracellular_output(
    cell_type_name="cancer_cell",
    physicell_name="apoptosis",
    intracellular_name="p53",
    action="activation",
    value=0.01,                   # Apoptosis rate when p53 is active
    base_value=0.0001,            # Background apoptosis rate
    smoothing=0
)

# Cell cycle regulation affects proliferation
config.physiboss.add_intracellular_output(
    cell_type_name="cancer_cell",
    physicell_name="cycle entry",
    intracellular_name="Rb",      # Rb protein controls G1/S transition
    action="activation",
    value=0.01,                   # Proliferation rate
    base_value=0.001,             # Background proliferation
    smoothing=0
)

# Survival signals affect migration
config.physiboss.add_intracellular_output(
    cell_type_name="cancer_cell",
    physicell_name="migration speed",
    intracellular_name="survival_signal",
    action="activation",
    value=2.0,                    # Enhanced migration when stressed
    base_value=0.8,               # Normal migration speed
    smoothing=0
)

print("   ✅ Added 3 output mappings")

# Add some mutations to simulate cancer progression
print("   🧬 Adding cancer-specific mutations...")

# Simulate p53 knockout (common in cancer)
config.physiboss.add_intracellular_mutation(
    cell_type_name="cancer_cell",
    intracellular_name="p53",
    value=False                   # p53 is knocked out
)

# Simulate oncogene activation
config.physiboss.add_intracellular_mutation(
    cell_type_name="cancer_cell",
    intracellular_name="oncogene",
    value=True                    # Oncogene is constitutively active
)

print("   ✅ Added cancer mutations (p53 knockout, oncogene activation)")

# Check if PhysiBoSS is enabled
if config.physiboss.is_enabled():
    print("\n✅ PhysiBoSS integration successfully configured!")
    print("   🔬 Boolean network will simulate intracellular gene regulation")
    print("   🔄 Network updates every 6 minutes during simulation")
    print("   📡 PhysiCell ↔ Boolean Network communication established")
else:
    print("❌ PhysiBoSS integration failed to enable")

🧬 Configuring PhysiBoSS/MaBoSS integration for cancer cells...
   ✅ MaBoSS model added to cancer cells
   🔗 Adding input mappings (PhysiCell → Boolean Network)...
   ✅ Added 3 input mappings
   🔗 Adding output mappings (Boolean Network → PhysiCell)...
   ✅ Added 3 output mappings
   🧬 Adding cancer-specific mutations...
   ✅ Added cancer mutations (p53 knockout, oncogene activation)

✅ PhysiBoSS integration successfully configured!
   🔬 Boolean network will simulate intracellular gene regulation
   🔄 Network updates every 6 minutes during simulation
   📡 PhysiCell ↔ Boolean Network communication established


## 6. Add Cell Rules for Behavior Control

Create cell rules that link environmental signals (like substrate concentrations) to cellular behaviors (like proliferation, death, migration).

In [7]:
print("📋 Adding cell rules for behavior control...")

# Get the cell rules CSV module (context-aware)
rules = config.cell_rules_csv

print("   🔍 Current context (available cell types and substrates):")
rules.print_context()

print("\n   📝 Adding cancer cell rules...")

# Cancer cell rules - oxygen affects behavior
# Low oxygen → necrosis (cell death)
rules.add_rule(
    cell_type="cancer_cell",
    signal="oxygen",
    direction="decreases",
    behavior="necrosis",
    base_value=0.0,      # No necrosis at high oxygen
    half_max=10.0,       # Half-maximum at 10 mmHg oxygen
    hill_power=4,        # Steep response curve
    apply_to_dead=0      # Don't apply to dead cells
)

# Oxygen promotes proliferation
rules.add_rule(
    cell_type="cancer_cell",
    signal="oxygen",
    direction="increases",
    behavior="cycle entry",
    base_value=0.001,    # Minimal proliferation without oxygen
    half_max=25.0,       # Half-maximum at 25 mmHg
    hill_power=2,        # Moderate steepness
    apply_to_dead=0
)

# Growth factor enhances proliferation
rules.add_rule(
    cell_type="cancer_cell",
    signal="growth_factor",
    direction="increases",
    behavior="cycle entry",
    base_value=0.001,
    half_max=0.5,        # Half-maximum at 0.5 ng/mL
    hill_power=3,
    apply_to_dead=0
)

# Glucose affects migration (seeking nutrients)
rules.add_rule(
    cell_type="cancer_cell",
    signal="glucose",
    direction="decreases",
    behavior="migration speed",
    base_value=0.2,      # Low migration with glucose
    half_max=2.0,        # Half-maximum at 2 mM glucose
    hill_power=2,
    apply_to_dead=0
)

# Apoptotic signal induces death
rules.add_rule(
    cell_type="cancer_cell",
    signal="apoptotic_signal",
    direction="increases",
    behavior="apoptosis",
    base_value=0.0001,   # Background apoptosis
    half_max=0.2,        # Sensitive to apoptotic signals
    hill_power=6,        # Very steep response
    apply_to_dead=0
)

print(f"   ✅ Added 5 cancer cell rules")

print("\n   📝 Adding immune cell rules...")

# Immune cell rules - contact with cancer cells activates them
rules.add_rule(
    cell_type="immune_cell",
    signal="contact with cancer_cell",
    direction="increases",
    behavior="attack cancer_cell",
    base_value=0.0,      # No attack without contact
    half_max=0.5,        # Activate quickly upon contact
    hill_power=8,        # Very sharp activation
    apply_to_dead=0
)

# Contact reduces migration (stop to attack)
rules.add_rule(
    cell_type="immune_cell",
    signal="contact with cancer_cell",
    direction="increases",
    behavior="migration speed",
    base_value=2.0,      # Normal speed
    half_max=0.3,        # Slow down upon contact
    hill_power=4,
    apply_to_dead=0
)

# Growth factor attracts immune cells
rules.add_rule(
    cell_type="immune_cell",
    signal="growth_factor",
    direction="increases",
    behavior="chemotactic response to growth_factor",
    base_value=0.0,
    half_max=0.8,
    hill_power=2,
    apply_to_dead=0
)

print(f"   ✅ Added 3 immune cell rules")

print("\n   📝 Adding stromal cell rules...")

# Stromal cells respond to tissue damage
rules.add_rule(
    cell_type="stromal_cell",
    signal="apoptotic_signal",
    direction="increases",
    behavior="growth_factor secretion",
    base_value=0.2,      # Baseline secretion
    half_max=0.1,        # Respond to damage signals
    hill_power=3,
    apply_to_dead=0
)

# Low oxygen affects stromal cell survival
rules.add_rule(
    cell_type="stromal_cell",
    signal="oxygen",
    direction="decreases",
    behavior="apoptosis",
    base_value=0.0001,
    half_max=5.0,        # More sensitive to hypoxia
    hill_power=4,
    apply_to_dead=0
)

print(f"   ✅ Added 2 stromal cell rules")

# Display all rules
print(f"\n📊 Total rules added: {len(rules.get_rules())}")
print("\n   📋 Rule summary:")
rules.print_rules()

# Add a ruleset to the main configuration
config.cell_rules.add_ruleset(
    name="main_rules",
    folder="./config",
    filename="cell_rules.csv",
    enabled=True
)

print("\n✅ Cell rules configuration completed!")

📋 Adding cell rules for behavior control...
   🔍 Current context (available cell types and substrates):

Current Context:
------------------------------
Cell Types: ['cancer_cell', 'immune_cell', 'stromal_cell']
Substrates: ['oxygen', 'glucose', 'growth_factor', 'apoptotic_signal']
Custom Variables: ['sample', 'somedata']

   📝 Adding cancer cell rules...
   ✅ Added 5 cancer cell rules

   📝 Adding immune cell rules...
   ✅ Added 3 immune cell rules

   📝 Adding stromal cell rules...
   ✅ Added 2 stromal cell rules

📊 Total rules added: 10

   📋 Rule summary:

Current Rules (10 total):
--------------------------------------------------------------------------------
#   Cell Type            Signal               Dir       Behavior             Base     Half     Hill  Dead
--------------------------------------------------------------------------------
0   cancer_cell          oxygen               decreases necrosis             0.0      10.0     4     0   
1   cancer_cell          oxygen  

In [8]:
config.cell_rules_csv.get_rules()

[{'cell_type': 'cancer_cell',
  'signal': 'oxygen',
  'direction': 'decreases',
  'behavior': 'necrosis',
  'base_value': 0.0,
  'half_max': 10.0,
  'hill_power': 4,
  'apply_to_dead': 0},
 {'cell_type': 'cancer_cell',
  'signal': 'oxygen',
  'direction': 'increases',
  'behavior': 'cycle entry',
  'base_value': 0.001,
  'half_max': 25.0,
  'hill_power': 2,
  'apply_to_dead': 0},
 {'cell_type': 'cancer_cell',
  'signal': 'growth_factor',
  'direction': 'increases',
  'behavior': 'cycle entry',
  'base_value': 0.001,
  'half_max': 0.5,
  'hill_power': 3,
  'apply_to_dead': 0},
 {'cell_type': 'cancer_cell',
  'signal': 'glucose',
  'direction': 'decreases',
  'behavior': 'migration speed',
  'base_value': 0.2,
  'half_max': 2.0,
  'hill_power': 2,
  'apply_to_dead': 0},
 {'cell_type': 'cancer_cell',
  'signal': 'apoptotic_signal',
  'direction': 'increases',
  'behavior': 'apoptosis',
  'base_value': 0.0001,
  'half_max': 0.2,
  'hill_power': 6,
  'apply_to_dead': 0},
 {'cell_type': 'imm

## 7. Export Cell Rules CSV and XML Configuration

Generate and save the cell rules CSV file and the complete PhysiCell XML configuration to disk.

In [9]:
print("💾 Exporting configuration files...")

# Create output directory
output_dir = "demo_output"
os.makedirs(output_dir, exist_ok=True)
print(f"   📁 Created output directory: {output_dir}")

# 1. Generate and save cell rules CSV file
csv_filename = os.path.join(output_dir, "cell_rules.csv")
try:
    rules.generate_csv(csv_filename)
    print(f"   ✅ Cell rules CSV saved: {csv_filename}")
    
    # Display the CSV contents
    print(f"\n   📄 Cell rules CSV contents:")
    with open(csv_filename, 'r') as f:
        lines = f.readlines()
        print(f"      📊 {len(lines)} rules exported")
        print("      📋 First 5 rules:")
        for i, line in enumerate(lines[:5], 1):
            print(f"         {i}: {line.strip()}")
        if len(lines) > 5:
            print(f"         ... and {len(lines) - 5} more rules")
            
except Exception as e:
    print(f"   ❌ Error generating CSV: {e}")

# 2. Configure save options for XML export
print(f"\n   ⚙️ Configuring save options...")
config.save_options.set_output_folder('./output')
config.save_options.set_full_data_options(interval=60.0, enable=True)
config.save_options.set_svg_options(interval=60.0, enable=True)

# Enable substrate visualization
config.save_options.set_svg_plot_substrate(
    enabled=True,
    substrate="oxygen",
    colormap="viridis",
    min_conc=0.0,
    max_conc=40.0
)

# Set cell coloring by cell type
config.save_options.set_svg_legend(enabled=True, cell_type=True)

print(f"   ✅ Save options configured")

# 3. Set initial conditions
config.initial_conditions.add_csv_file(
    filename="cells.csv",
    folder="./config",
    enabled=True
)

# Set user parameters
config.add_user_parameter("random_seed", 42, "dimensionless", "Random seed for reproducibility", "int")
config.set_number_of_cells(0)  # Using CSV file for initial conditions

print(f"   ✅ Initial conditions and user parameters set")

# 4. Generate and save PhysiCell XML configuration
xml_filename = os.path.join(output_dir, "PhysiCell_settings.xml")
try:
    config.save_xml(xml_filename)
    print(f"   ✅ PhysiCell XML saved: {xml_filename}")
    
    # Display XML file size and basic info
    file_size = os.path.getsize(xml_filename)
    print(f"      📏 File size: {file_size:,} bytes ({file_size/1024:.1f} KB)")
    
    # Count XML elements
    with open(xml_filename, 'r') as f:
        xml_content = f.read()
        
    element_counts = {
        'substrates': xml_content.count('<variable'),
        'cell_types': xml_content.count('<cell_definition'),
        'user_parameters': xml_content.count('</user_parameters>') > 0
    }
    
    print(f"      📊 XML contains:")
    print(f"         🧪 {element_counts['substrates']} substrates")
    print(f"         🦠 {element_counts['cell_types']} cell types")
    print(f"         ⚙️ User parameters: {'Yes' if element_counts['user_parameters'] else 'No'}")
    
    # Check for PhysiBoSS elements
    physiboss_count = xml_content.count('<intracellular')
    if physiboss_count > 0:
        print(f"         🧬 {physiboss_count} PhysiBoSS intracellular models")
    
except Exception as e:
    print(f"   ❌ Error generating XML: {e}")

# 5. Create a summary file
summary_filename = os.path.join(output_dir, "configuration_summary.json")
try:
    summary = config.get_summary()
    summary['export_timestamp'] = datetime.now().isoformat()
    summary['files_exported'] = {
        'cell_rules_csv': csv_filename,
        'physicell_xml': xml_filename
    }
    
    with open(summary_filename, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"   ✅ Configuration summary saved: {summary_filename}")
    
except Exception as e:
    print(f"   ❌ Error creating summary: {e}")

print(f"\n✅ Export completed successfully!")
print(f"📁 All files saved in: {os.path.abspath(output_dir)}")
print(f"🎯 Ready for PhysiCell simulation!")

💾 Exporting configuration files...
   📁 Created output directory: demo_output
Generated cell rules CSV: demo_output/cell_rules.csv
   ✅ Cell rules CSV saved: demo_output/cell_rules.csv

   📄 Cell rules CSV contents:
      📊 10 rules exported
      📋 First 5 rules:
         1: cancer_cell,oxygen,decreases,necrosis,0.0,10.0,4,0
         2: cancer_cell,oxygen,increases,cycle entry,0.001,25.0,2,0
         3: cancer_cell,growth_factor,increases,cycle entry,0.001,0.5,3,0
         4: cancer_cell,glucose,decreases,migration speed,0.2,2.0,2,0
         5: cancer_cell,apoptotic_signal,increases,apoptosis,0.0001,0.2,6,0
         ... and 5 more rules

   ⚙️ Configuring save options...
   ✅ Save options configured
   ✅ Initial conditions and user parameters set
   ✅ PhysiCell XML saved: demo_output/PhysiCell_settings.xml
      📏 File size: 34,470 bytes (33.7 KB)
      📊 XML contains:
         🧪 4 substrates
         🦠 4 cell types
         ⚙️ User parameters: Yes
         🧬 2 PhysiBoSS intracellular

## 8. Validate and Summary Check

Validate the configuration for any issues and display a comprehensive summary of all configured components.

In [10]:
print("✅ Validating configuration and generating summary...")

# 1. Validate the configuration
print("\n🔍 Configuration Validation:")
validation_issues = config.validate()

if validation_issues:
    print("   ⚠️ Validation issues found:")
    for issue in validation_issues:
        print(f"      • {issue}")
else:
    print("   ✅ Configuration is valid - no issues found!")

# 2. Validate cell rules specifically
print("\n🔍 Cell Rules Validation:")
rules_validation = rules.validate_rules()

if rules_validation:
    print("   ⚠️ Cell rules issues:")
    for issue in rules_validation:
        print(f"      • {issue}")
else:
    print("   ✅ All cell rules are valid!")

# 3. Generate comprehensive summary
print("\n📊 Configuration Summary:")
summary = config.get_summary()

print(f"   🏗️ Domain:")
print(f"      📏 Bounds: X=[{summary['domain']['x_min']}, {summary['domain']['x_max']}], "
      f"Y=[{summary['domain']['y_min']}, {summary['domain']['y_max']}]")
print(f"      🔲 Mesh: {summary['domain']['dx']} × {summary['domain']['dy']} × {summary['domain']['dz']} μm")
print(f"      📐 2D mode: {summary['domain']['use_2D']}")

print(f"\n   🧪 Substrates ({len(summary['substrates'])}):")
for substrate in summary['substrates']:
    substrate_info = config.substrates.get_substrates()[substrate]
    print(f"      • {substrate}: D={substrate_info['diffusion_coefficient']:.0f} μm²/min, "
          f"decay={substrate_info['decay_rate']:.3f} min⁻¹")

print(f"\n   🦠 Cell Types ({len(summary['cell_types'])}):")
for cell_type in summary['cell_types']:
    cell_info = config.cell_types.get_cell_types()[cell_type]
    cycle_model = cell_info['phenotype']['cycle'].get('model', 'default')
    has_intracellular = 'intracellular' in cell_info['phenotype']
    print(f"      • {cell_type}: cycle={cycle_model}, PhysiBoSS={'Yes' if has_intracellular else 'No'}")

print(f"\n   📋 Cell Rules:")
print(f"      • Total rules: {len(rules.get_rules())}")
print(f"      • Rulesets: {summary['num_rulesets']}")

print(f"\n   🧬 PhysiBoSS Integration:")
print(f"      • Enabled: {summary['physiboss_enabled']}")
if summary['physiboss_enabled']:
    cancer_cell = config.cell_types.get_cell_types()['cancer_cell']
    if 'intracellular' in cancer_cell['phenotype']:
        intracellular = cancer_cell['phenotype']['intracellular']
        num_inputs = len(intracellular.get('mapping', {}).get('inputs', []))
        num_outputs = len(intracellular.get('mapping', {}).get('outputs', []))
        num_mutations = len(intracellular.get('settings', {}).get('mutations', []))
        print(f"      • Input mappings: {num_inputs}")
        print(f"      • Output mappings: {num_outputs}")
        print(f"      • Mutations: {num_mutations}")

print(f"\n   ⚙️ Simulation Options:")
options = summary['options']
print(f"      • Max time: {options['max_time']} minutes ({options['max_time']/60:.1f} hours)")
print(f"      • Time steps: diffusion={options['dt_diffusion']}, mechanics={options['dt_mechanics']}, phenotype={options['dt_phenotype']}")
print(f"      • Parallel threads: {options['omp_num_threads']}")

print(f"\n   🎮 User Parameters ({len(summary['user_parameters'])}):")
for param in summary['user_parameters']:
    param_info = config.user_parameters[param]
    print(f"      • {param}: {param_info['value']} {param_info['units']} ({param_info['description']})")

print(f"\n   💾 Output Configuration:")
save_opts = summary['save_options']
print(f"      • Output folder: {save_opts.get('folder', 'N/A')}")
print(f"      • Full data: {save_opts.get('full_data', {}).get('enabled', False)} "
      f"(interval: {save_opts.get('full_data', {}).get('interval', 'N/A')} min)")
print(f"      • SVG plots: {save_opts.get('svg', {}).get('enabled', False)} "
      f"(interval: {save_opts.get('svg', {}).get('interval', 'N/A')} min)")

# 4. Performance estimates
print(f"\n⚡ Performance Estimates:")
domain = summary['domain']
domain_volume = (domain['x_max'] - domain['x_min']) * (domain['y_max'] - domain['y_min']) * (domain['z_max'] - domain['z_min'])
voxel_volume = domain['dx'] * domain['dy'] * domain['dz']
num_voxels = int(domain_volume / voxel_volume)

print(f"   📊 Domain statistics:")
print(f"      • Total volume: {domain_volume:,.0f} μm³")
print(f"      • Number of voxels: {num_voxels:,}")
print(f"      • Voxel volume: {voxel_volume:.0f} μm³")

simulation_minutes = options['max_time']
phenotype_steps = int(simulation_minutes / options['dt_phenotype'])
diffusion_steps = int(simulation_minutes / options['dt_diffusion'])

print(f"   ⏱️ Simulation steps:")
print(f"      • Phenotype updates: {phenotype_steps:,}")
print(f"      • Diffusion steps: {diffusion_steps:,}")

print(f"\n🎯 Configuration Complete!")
print(f"📈 This simulation will model {len(summary['cell_types'])} cell types in a {len(summary['substrates'])}-substrate microenvironment")
print(f"🧬 PhysiBoSS integration will simulate intracellular gene regulation in cancer cells")
print(f"📋 {len(rules.get_rules())} cell rules will control cell behavior based on environmental conditions")
print(f"🏁 Ready for PhysiCell simulation!")

✅ Validating configuration and generating summary...

🔍 Configuration Validation:
   ✅ Configuration is valid - no issues found!

🔍 Cell Rules Validation:
   ✅ All cell rules are valid!

📊 Configuration Summary:
   🏗️ Domain:
      📏 Bounds: X=[-400.0, 400.0], Y=[-400.0, 400.0]
      🔲 Mesh: 20.0 × 20.0 × 20.0 μm
      📐 2D mode: True

   🧪 Substrates (4):
      • oxygen: D=100000 μm²/min, decay=0.100 min⁻¹
      • glucose: D=50000 μm²/min, decay=0.005 min⁻¹
      • growth_factor: D=25000 μm²/min, decay=0.010 min⁻¹
      • apoptotic_signal: D=10000 μm²/min, decay=0.050 min⁻¹

   🦠 Cell Types (3):
      • cancer_cell: cycle=Ki67_basic, PhysiBoSS=Yes
      • immune_cell: cycle=live, PhysiBoSS=No
      • stromal_cell: cycle=cycling_quiescent, PhysiBoSS=No

   📋 Cell Rules:
      • Total rules: 10
      • Rulesets: 1

   🧬 PhysiBoSS Integration:
      • Enabled: True
      • Input mappings: 3
      • Output mappings: 3
      • Mutations: 2

   ⚙️ Simulation Options:
      • Max time: 2880.

In [11]:
config.get_summary()

{'domain': {'x_min': -400.0,
  'x_max': 400.0,
  'y_min': -400.0,
  'y_max': 400.0,
  'z_min': -10.0,
  'z_max': 10.0,
  'dx': 20.0,
  'dy': 20.0,
  'dz': 20.0,
  'use_2D': True},
 'substrates': ['oxygen', 'glucose', 'growth_factor', 'apoptotic_signal'],
 'cell_types': ['cancer_cell', 'immune_cell', 'stromal_cell'],
 'user_parameters': ['number_of_cells', 'random_seed'],
 'num_rules': 10,
 'num_rulesets': 1,
 'physiboss_enabled': True,
 'initial_conditions': 4,
 'options': {'virtual_wall_at_domain_edge': True,
  'disable_automated_spring_adhesions': False,
  'legacy_random_points_on_sphere_in_divide': False,
  'random_seed': 0,
  'max_time': 2880.0,
  'time_units': 'min',
  'space_units': 'micron',
  'dt_diffusion': 0.01,
  'dt_mechanics': 0.1,
  'dt_phenotype': 6.0,
  'omp_num_threads': 4},
 'save_options': {'folder': './output',
  'full_data': {'interval': 60.0, 'enable': True},
  'SVG': {'interval': 60.0,
   'enable': True,
   'plot_substrate': {'enabled': True,
    'limits': False,

## 🎉 Demo Complete!

### What We Accomplished

This notebook demonstrated the comprehensive capabilities of the **PhysiCell Settings** package:

1. **🏗️ Domain Configuration** - Set up a 2D simulation domain with appropriate bounds and mesh spacing
2. **🧪 Substrate Management** - Added 4 different substrates (oxygen, glucose, growth factor, apoptotic signal)
3. **🦠 Cell Type Definition** - Created 3 cell types with distinct properties:
   - **Cancer cells**: Ki67 cycle model with PhysiBoSS integration
   - **Immune cells**: Live cycle with high motility
   - **Stromal cells**: Slow cycling support cells
4. **🧬 PhysiBoSS Integration** - Configured MaBoSS boolean networks for cancer cells with:
   - Input mappings (PhysiCell → Boolean network)
   - Output mappings (Boolean network → PhysiCell)
   - Cancer-specific mutations (p53 knockout, oncogene activation)
5. **📋 Cell Rules** - Created 10+ behavioral rules linking environmental signals to cellular responses
6. **💾 File Export** - Generated PhysiCell-compatible files:
   - `cell_rules.csv` - Behavioral rules for PhysiCell
   - `PhysiCell_settings.xml` - Complete simulation configuration

### Next Steps

To run this simulation in PhysiCell:

1. **Copy the exported files** from `demo_output/` to your PhysiCell project directory
2. **Create the boolean network files** referenced in the PhysiBoSS configuration:
   - `config/cancer_network.bnd` - Boolean network definition
   - `config/cancer_network.cfg` - Network parameters
3. **Create initial conditions file** `config/cells.csv` with starting cell positions
4. **Compile and run PhysiCell** with the generated `PhysiCell_settings.xml`

### Package Features Demonstrated

✅ **Modular Architecture** - Clean separation of concerns across modules  
✅ **PhysiBoSS Integration** - Boolean network support for intracellular modeling  
✅ **Cell Rules System** - Context-aware CSV generation for behavioral rules  
✅ **Comprehensive Validation** - Built-in configuration checking  
✅ **Export Capabilities** - Multiple output formats  
✅ **Flexible Configuration** - Easy parameter adjustment and extension  

### Resources

- **Package**: `pip install physicell-settings`
- **Repository**: https://github.com/mruscone/PhysiCell_Settings  
- **PyPI**: https://pypi.org/project/physicell-settings/
- **PhysiCell**: http://physicell.org/

---

**Thank you for exploring the PhysiCell Settings package!** 🚀